# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'./movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [3]:
from sklearn.externals import joblib
eclf = joblib.load('sentiment.pkl') 

IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [4]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [5]:
wml_credentials={
  "apikey": "ecLhS-sB8eN4hYbaMZJiG7G9o9tVOBHwKkHakrbzNuVm",
  "iam_apikey_description": "Auto-generated for key b5870580-962a-45c7-86ac-64bfc9879a3b",
  "iam_apikey_name": "Credenciales de servicio-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/9fdd614649ae4cac90110a30b8dbcda9::serviceid:ServiceId-11378b29-d8ca-4f7e-b6ed-b637e3243800",
  "instance_id": "4d49298d-3f83-41e6-a5b4-9076cc779b94",
  "url": "https://us-south.ml.cloud.ibm.com"
}

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [6]:
client=WatsonMachineLearningAPIClient(wml_credentials)

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [7]:
model_props={client.repository.ModelMetaNames.AUTHOR_NAME: "Gabriela Rau", 
               client.repository.ModelMetaNames.NAME: "Modelo_Entrega7",
               client.repository.DefinitionMetaNames.RUNTIME_NAME: 'python',
               client.repository.DefinitionMetaNames.RUNTIME_VERSION: '3.6'}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [13]:
#librería tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
#librería papeline
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

tfidf_vect=TfidfVectorizer()

In [14]:
pipeline=Pipeline([('tfidf_vect',tfidf_vect),('eclf',eclf)])
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf_vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...
                                                                                                        max_depth=2,
                                                                                                        max_features=None,
                                                              

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [15]:
published_model=client.repository.store_model(model=pipeline, 
                                                meta_props=model_props, 
                                                training_data=X_train, 
                                                training_target=y_train)

**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [16]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

In [17]:
models_details=client.repository.list_models()

------------------------------------  ------------------------  ------------------------  -----------------
GUID                                  NAME                      CREATED                   FRAMEWORK
fc15d03b-cad0-4404-8225-8ad12c249cab  Modelo_Entrega7           2020-05-12T01:59:43.475Z  scikit-learn-0.22
719b5f76-6536-41a3-9c5f-3d932f7d1482  Modelo_Entrega7           2020-05-07T23:48:46.604Z  scikit-learn-0.22
79b83dc1-559f-4f79-8244-17752997b04e  Modelo_Entrega7           2020-05-07T23:42:07.376Z  scikit-learn-0.22
87f1027c-498a-4ca4-afa5-8725b0d636f8  Modelo_Entrega7           2020-05-07T23:38:51.157Z  scikit-learn-0.22
b69897ca-06fa-4389-ae15-6f40bbcf35a6  Modelo_Entrega7           2020-05-07T23:38:30.018Z  scikit-learn-0.22
28b60cd9-c9e4-4a4b-bada-6a1a0baa937e  Primer modelo en la nube  2020-05-07T04:16:07.342Z  scikit-learn-0.22
------------------------------------  ------------------------  ------------------------  -----------------


**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [18]:
loaded_model = client.repository.load(published_model_uid)
test_predictions = loaded_model.predict(X_test)

**9) Mostrar** el `classification_report` obtenido

In [19]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

print("roc_auc score: ", roc_auc_score(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

roc_auc score:  0.8575
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       200
           1       0.90      0.80      0.85       200

    accuracy                           0.86       400
   macro avg       0.86      0.86      0.86       400
weighted avg       0.86      0.86      0.86       400

